# Step 18, 메모리 절약 모드

DeZero의 메모리 사용을 개선할 수 있는 구조 두 가지를 도입힌다.  
1. 역전파 시 사용하는 메모리양을 줄이는 방법, 불필요한 미분 결과를 보관하지 않고 즉시 삭제  
2. '역전파가 필요없는 경우용 모드'를 제공, 불필요한 계산을 생략

## 18.1 필요 없는 미분값 삭제  

In [3]:
import numpy as np
import weakref

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.


def as_array(x):
    if np.isscalar(x):
        return np.array(x)      # 스칼라이면 array로 바꿔서 리턴
    return x                    # 스칼라 아니면(array이면) 그대로 리턴


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        #self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)



In [4]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


y.backward()를 실행하여 미분하면 '모든 변수'가 미분 결과를 메모리에 유지  
그러나 역전파로 구하고 싶은 미분값은 말단 변수(x0,x1)뿐이다. (대부분 머신러닝에서)  
즉, 중간 변수의 미분값은 필요하지 않다.  

그래서 중간 변수에 대해서는 미분값을 제거하는 모드를 추가한다.

~~~python 
class Variable:
    # ... 생략 ...
    def backward(self, retain_grad=False):

    # .. 생략 ..
    if not retain_grad:
        for y in f.outputs:
            y().grad = None     # y는 약한 참조
~~~

In [6]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None                        # 미분값 초기화

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None 

retain_grad=True : 지금까지처럼 모든 변수가 미분 결과(기울기)를 유지한다.  
retain_grad=False(기본값) : 중간 변수의 미분값을 모두 None으로 재설정  

In [7]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


이와 같이 중간변수인 y와t의 미분값이 삭제되어 그만큼의 메모리를 다른 용도로 사용할수 있게 된다.

다음은 두 번째 개선 차례지만, Function 클래스를 복습해본다.

## 18.2 Function 클래스 복습

미분을 하려면 순전파를 수행한 뒤 역전파  
역전파 시에는 순전파의 계산 결과가 필요하기 때문에 순전파 때 결과값을 기억해둔다.  

결과값을 보관하는 로직은 
~~~python
class Function:
    def __call__(self, *inputs):

        # ... 생략 ...

        self.inputs = inputs
        
        # ... 생략 ....
~~~

이와 같은 함수는 입력을 inputs라는 '인스턴스 변수'로 참조한다.  
그 결과 inputs가 참조하는 변수의 참조 카운트가 1만큼 증가  
__call__ 메서드에서 벗어난 뒤에도 메모리에 생존한다. 

인스턴스 변수 inputs는 역전파 계산시 사용된다. --> 역전파를 하는 경우라면 참조할 변수들을 inputs에 미리 보관해둬야 한다.  
하지만 미분값이 필요 없는 경우 중간 계산 결과를 저장할 필요가 없고, 계산의 '연결' 또한 만들 필요가 없다.

**즉, 신경망에서**    
학습 시에는 미분값을 구해야하지만  
추론 시에는 단순히 순전파만 하기 때문에 중간 계산 결과를 곧바로 버려 메모리 사용량을 크게 줄여야 한다.  

(신경망에는 학습(훈련)과 추론 두가지 단계가 있음))


## 18.3 Config 클래스를 활용한 모드 전환 

순전파만 할 경우를 위해 모드 개선  
1. '역전파 활성 모드'  
2. '역전파 비활성 모드'

In [9]:
class Config:
    enable_backprop = True 

enable_backprop는 역전파가 가능한지 여부를 뜻하고 이 값이 True면 '역전파 활성 모드'이다.

**CAUTION_**  
설정 데이터는 단 한군데에만 존재하는것이 좋다. ==> Config 클래스는 '인스턴스'화 하지않고 '클래스'상태로 이용  
'인스턴스'는 여러 개 생성할 수 있지만 클래스는 항상 하나만 존재하기 때문  
Config 클래스가 '클래스 속성'을 갖도록 설정

In [11]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        if Config.enable_backprop:                          # 추가 변경된 부분   
            self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

            for output in outputs:
                output.set_creator(self)              # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

            self.inputs = inputs
            #self.outputs = outputs
            self.outputs = [weakref.ref(output) for output in outputs]
            
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

이와 같이 Config.enable_backprop이 True일 때만 역전파 코드가 실행 

## 18.4 모드 전환 

In [12]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()                        # True이면 중간 계산 결과가 (역전파가 완료되기 전까지는)계속 유지되어 메모리를 차지 

Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
                                    # False이면 중간 계산 결과는 사용 후 곧바로 삭제된다.

## 18.5 with문을 활용한 모드 전환 

with문 : 후처리를 자동으로 수행  

with문 없이 파일에 무언가를 쓸때 

~~~python
f = open('sample.txt', 'w')
f.write('hello')
f.close()
~~~

close하기 귀찮을때 

~~~python
with open('sample.txt', 'w') as f:
    f.write('hello world!')
~~~

with 블록에 들어갈 때 파일이 열림  
with 블록 안에서 파일은 계속 열린 상태  
with 블록에 빠져나올때 자동으로 닫힌다.

즉 with문을 사용하는 것으로  
'with 블록에 들어갈 때의 처리(전처리)'  
'with 블록을 빠져나 올 때의 처리(후처리)'  
를 자동으로 할 수 있다.

이러한 with문의 원리를 이용하여 '역전파 비활성 모드'로 전환

~~~python 
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
~~~
이와 같이 with using_config('enable_backprop', False): 안에서만 '역전파 비활겅 모드'가 된다.  
벗어나면 일반모드 : '역전파 활성 모드'

**NOTE_** '역전파 비활성 모드'로 일시적으로 전환하는 방법은 실전에서 자주 사용됨  
신경망 학습에서는 모델 평가를 (학습 도중에) 하기 위해 기울기가 필요없는 모드를 사용하는 일이 자주 발생

with 문을 사용한 모드 전환 구현  

contextlib 모듈을 사용하면 가장 쉽게 구현할 수 있다.

먼저 contextlib 모듈 사용법

In [14]:
import contextlib 

@contextlib.contextmanager
def config_test():
    print('start')  # 전처리 
    try:
        yield
    finally:
        print('done')

with config_test():
    print('process....')

start
process....
done


**@contextlib.contextmanager 데코레이터** 를 달면 문맥context를 판단하는 함수가 만들어진다.

이 함수 안에서 yield 전에는 전처리 로직을  
yield 다음에는 후처리 로직을 작성  

with config_test(): 형태의 구문을 사용할 수 있다.  
이 구문을 사용하면 with 블록 안으로 들어갈 때 전처리가 실행되고   
블록 범위를 빠져나올 때 후처리가 실행된다.

In [15]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):          # name은 사용할 Config 속성의 이름(클래스 속성 이름), 타입은 str 
    old_value = getattr(Config, name)   # name을 getarr 함수에 넘겨 Config 클래스에서 꺼내온다.
    setattr(Config, name, value)        # setarr 함수를 사용하여 새로운 값을 설정 
    try:
        yield
    finally:
        setattr(Config, name, old_value)

name은 사용할 Config 속성의 이름(클래스 속성 이름), 타입은 str  
name을 getarr 함수에 넘겨 Config 클래스에서 꺼내온다.  
setarr 함수를 사용하여 새로운 값을 설정

이제 with 블록에 들어갈 때 : name으로 지정한 Config 클래스 속성이 value로 설정된다.  
with 블록을 빠져나올 때 : 원래 값(old_value)으로 복원된다.

In [16]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

이와 같이 역전파가 필요 없는 경우에는 with 블록에서 순전파 코드만 실행한다.  
이제 불필요한 계산을 생략하고 메모리를 절약할 수 있다.

그러나 using_config('enable_backprop', False): 라는 긴 코드를 매번 적어주기 귀찮 --> no_grad 함수를 만든다.

In [17]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

이제 기울기가 필요없을때는 no_grad 함수를 호출하면 된다.